# hHGTN Fraud Detection Demo API Client

This notebook demonstrates how to interact with the hHGTN fraud detection demo service using Python.
It shows how to make API calls, visualize results, and render explanation graphs.

## Setup and Dependencies

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
try:
    import networkx as nx
    import matplotlib.pyplot as plt
    from pyvis.network import Network
    VISUALIZATION_AVAILABLE = True
except ImportError:
    print("Note: Install networkx, matplotlib, and pyvis for full visualization features")
    print("pip install networkx matplotlib pyvis")
    VISUALIZATION_AVAILABLE = False

# Demo service configuration
API_BASE_URL = "http://localhost:8000"
print(f"Demo API Base URL: {API_BASE_URL}")

## API Helper Functions

In [ ]:
def check_service_health():
    """Check if the demo service is running and healthy."""
    try:
        response = requests.get(f"{API_BASE_URL}/health", timeout=5)
        response.raise_for_status()
        health_data = response.json()
        print(f"✅ Service Status: {health_data['status']}")
        print(f"📊 Model Loader: {health_data['model_loader_status']}")
        print(f"⏰ Uptime: {health_data['uptime_seconds']:.1f} seconds")
        return True
    except Exception as e:
        print(f"❌ Service not available: {e}")
        print("Make sure the demo service is running with: uvicorn app:app --reload")
        return False

def predict_fraud(transaction_data, explain_config=None):
    """Send a transaction for fraud prediction."""
    if explain_config is None:
        explain_config = {
            "top_k_nodes": 15,
            "top_k_edges": 20
        }
    
    payload = {
        "transaction": transaction_data,
        "explain_config": explain_config
    }
    
    try:
        response = requests.post(
            f"{API_BASE_URL}/predict", 
            json=payload,
            headers={"Content-Type": "application/json"},
            timeout=30
        )
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Prediction request failed: {e}")
        if hasattr(e, 'response') and e.response is not None:
            try:
                error_detail = e.response.json()
                print(f"Error details: {error_detail}")
            except:
                print(f"Response text: {e.response.text}")
        return None

def get_service_metrics():
    """Get service performance metrics."""
    try:
        response = requests.get(f"{API_BASE_URL}/metrics", timeout=5)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ Could not fetch metrics: {e}")
        return None

## Check Service Status

In [ ]:
# Verify the demo service is running
service_available = check_service_health()

if service_available:
    print("\n🚀 Ready to test fraud detection!")
else:
    print("\n⚠️  Please start the demo service first:")
    print("   cd demo_service")
    print("   uvicorn app:app --reload")

## Sample Transaction Data

In [ ]:
# Sample legitimate transaction
legitimate_transaction = {
    "user_id": "user_regular_123",
    "merchant_id": "merchant_trusted_456",
    "device_id": "device_known_abc",
    "ip_address": "192.168.1.100",
    "timestamp": datetime.now().isoformat(),
    "amount": 45.99,
    "currency": "USD",
    "location": "New York, NY",
    "context": {
        "merchant_category": "grocery",
        "is_weekend": False
    }
}

# Sample suspicious transaction
suspicious_transaction = {
    "user_id": "user_suspicious_999",
    "merchant_id": "merchant_highrisk_666",
    "device_id": "device_unknown_xyz",
    "ip_address": "123.45.67.89",
    "timestamp": (datetime.now() - timedelta(hours=2)).isoformat(),
    "amount": 9999.99,
    "currency": "USD",
    "location": "Unknown Location",
    "context": {
        "merchant_category": "cash_advance",
        "is_weekend": True
    }
}

print("📝 Sample transactions prepared:")
print(f"1. Legitimate: ${legitimate_transaction['amount']} at {legitimate_transaction['context']['merchant_category']}")
print(f"2. Suspicious: ${suspicious_transaction['amount']} at {suspicious_transaction['context']['merchant_category']}")

## Test Fraud Prediction - Legitimate Transaction

In [ ]:
if service_available:
    print("🔍 Analyzing legitimate transaction...\n")
    
    result = predict_fraud(legitimate_transaction)
    
    if result:
        print(f"📊 PREDICTION RESULTS:")
        print(f"   Label: {result['predicted_label'].upper()}")
        print(f"   Fraud Probability: {result['prediction_prob']:.1%}")
        print(f"   Confidence: {result['confidence']:.1%}")
        print(f"   Model Version: {result['meta']['model_version']}")
        print(f"   Analysis Time: {result['meta']['explain_time_ms']} ms")
        
        if 'explanation' in result:
            exp = result['explanation']
            print(f"\n🕸️ EXPLANATION GRAPH:")
            print(f"   Nodes: {len(exp['nodes'])} entities")
            print(f"   Edges: {len(exp['edges'])} relationships")
            print(f"   Top Features: {len(exp['top_features'])} key indicators")
        
        # Store result for visualization
        legit_result = result
else:
    print("⚠️ Service not available - skipping prediction test")

## Test Fraud Prediction - Suspicious Transaction

In [ ]:
if service_available:
    print("🚨 Analyzing suspicious transaction...\n")
    
    result = predict_fraud(suspicious_transaction)
    
    if result:
        print(f"📊 PREDICTION RESULTS:")
        print(f"   Label: {result['predicted_label'].upper()}")
        print(f"   Fraud Probability: {result['prediction_prob']:.1%}")
        print(f"   Confidence: {result['confidence']:.1%}")
        print(f"   Model Version: {result['meta']['model_version']}")
        print(f"   Analysis Time: {result['meta']['explain_time_ms']} ms")
        
        if 'explanation' in result:
            exp = result['explanation']
            print(f"\n🕸️ EXPLANATION GRAPH:")
            print(f"   Nodes: {len(exp['nodes'])} entities")
            print(f"   Edges: {len(exp['edges'])} relationships")
            print(f"   Top Features: {len(exp['top_features'])} key indicators")
            
            print(f"\n🔝 TOP RISK FACTORS:")
            for i, feature in enumerate(exp['top_features'][:5], 1):
                print(f"   {i}. {feature['feature_name']}: {feature['importance_score']:.1%} importance")
        
        # Store result for visualization
        fraud_result = result
else:
    print("⚠️ Service not available - skipping prediction test")

## Visualization Functions

In [ ]:
def create_explanation_graph(explanation_data, title="Explanation Graph"):
    """Create a NetworkX graph from explanation data."""
    if not VISUALIZATION_AVAILABLE:
        print("⚠️ Visualization libraries not available")
        return None
        
    G = nx.Graph()
    
    # Add nodes with attributes
    for node in explanation_data['nodes']:
        G.add_node(
            node['id'], 
            type=node['type'],
            importance=node['importance_score'],
            label=node['id'].split('_')[0]  # Simplified label
        )
    
    # Add edges with attributes
    for edge in explanation_data['edges']:
        G.add_edge(
            edge['source'], 
            edge['target'],
            relation=edge['relation_type'],
            importance=edge['importance_score']
        )
    
    return G

def plot_explanation_graph(G, title="Explanation Graph", figsize=(12, 8)):
    """Plot explanation graph with matplotlib."""
    if not VISUALIZATION_AVAILABLE or G is None:
        return
        
    plt.figure(figsize=figsize)
    
    # Node positioning
    pos = nx.spring_layout(G, k=3, iterations=50)
    
    # Color scheme for node types
    type_colors = {
        'user': '#e74c3c',
        'merchant': '#3498db', 
        'device': '#f39c12',
        'ip': '#2ecc71'
    }
    
    # Draw nodes by type
    for node_type, color in type_colors.items():
        nodes_of_type = [n for n, d in G.nodes(data=True) if d.get('type') == node_type]
        if nodes_of_type:
            node_sizes = [300 + G.nodes[n]['importance'] * 500 for n in nodes_of_type]
            nx.draw_networkx_nodes(
                G, pos, nodelist=nodes_of_type, 
                node_color=color, node_size=node_sizes,
                alpha=0.8, label=node_type.title()
            )
    
    # Draw edges with varying thickness
    edge_weights = [G[u][v]['importance'] * 3 for u, v in G.edges()]
    nx.draw_networkx_edges(
        G, pos, width=edge_weights, alpha=0.6, edge_color='gray'
    )
    
    # Draw labels
    labels = {n: G.nodes[n]['label'] for n in G.nodes()}
    nx.draw_networkx_labels(G, pos, labels, font_size=8)
    
    plt.title(title, size=16, weight='bold')
    plt.legend(loc='upper right')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def create_interactive_graph(explanation_data, filename="explanation_graph.html"):
    """Create an interactive graph using pyvis."""
    if not VISUALIZATION_AVAILABLE:
        print("⚠️ Pyvis not available for interactive graphs")
        return None
        
    net = Network(height="500px", width="100%", bgcolor="#f8f9fa", font_color="black")
    
    # Color scheme
    type_colors = {
        'user': '#e74c3c',
        'merchant': '#3498db', 
        'device': '#f39c12',
        'ip': '#2ecc71'
    }
    
    # Add nodes
    for node in explanation_data['nodes']:
        net.add_node(
            node['id'],
            label=node['id'].split('_')[0],
            color=type_colors.get(node['type'], '#95a5a6'),
            size=20 + node['importance_score'] * 30,
            title=f"Type: {node['type']}\nImportance: {node['importance_score']:.1%}"
        )
    
    # Add edges
    for edge in explanation_data['edges']:
        net.add_edge(
            edge['source'], 
            edge['target'],
            width=1 + edge['importance_score'] * 5,
            title=f"Relation: {edge['relation_type']}\nImportance: {edge['importance_score']:.1%}"
        )
    
    # Configure physics
    net.set_options("""
    var options = {
      "physics": {
        "enabled": true,
        "stabilization": {"iterations": 100}
      }
    }
    """)
    
    net.save_graph(filename)
    print(f"💾 Interactive graph saved as {filename}")
    return filename

## Visualize Results

In [ ]:
if service_available and VISUALIZATION_AVAILABLE:
    # Visualize legitimate transaction explanation
    if 'legit_result' in locals() and legit_result:
        print("📊 Visualizing legitimate transaction explanation...")
        legit_graph = create_explanation_graph(
            legit_result['explanation'], 
            "Legitimate Transaction - Explanation Graph"
        )
        if legit_graph:
            plot_explanation_graph(legit_graph, "Legitimate Transaction Analysis")
            create_interactive_graph(
                legit_result['explanation'], 
                "legitimate_explanation.html"
            )
else:
    print("⚠️ Skipping visualization - service not available or libraries missing")

In [ ]:
if service_available and VISUALIZATION_AVAILABLE:
    # Visualize suspicious transaction explanation
    if 'fraud_result' in locals() and fraud_result:
        print("🚨 Visualizing suspicious transaction explanation...")
        fraud_graph = create_explanation_graph(
            fraud_result['explanation'], 
            "Suspicious Transaction - Explanation Graph"
        )
        if fraud_graph:
            plot_explanation_graph(fraud_graph, "Suspicious Transaction Analysis")
            create_interactive_graph(
                fraud_result['explanation'], 
                "suspicious_explanation.html"
            )
else:
    print("⚠️ Skipping visualization - service not available or libraries missing")

## Comparative Analysis

In [ ]:
if service_available and 'legit_result' in locals() and 'fraud_result' in locals():
    print("📈 COMPARATIVE ANALYSIS\n")
    
    # Create comparison dataframe
    comparison_data = {
        'Metric': ['Predicted Label', 'Fraud Probability', 'Confidence', 'Analysis Time (ms)', 'Graph Nodes', 'Graph Edges'],
        'Legitimate Transaction': [
            legit_result['predicted_label'],
            f"{legit_result['prediction_prob']:.1%}",
            f"{legit_result['confidence']:.1%}",
            legit_result['meta']['explain_time_ms'],
            len(legit_result['explanation']['nodes']),
            len(legit_result['explanation']['edges'])
        ],
        'Suspicious Transaction': [
            fraud_result['predicted_label'],
            f"{fraud_result['prediction_prob']:.1%}",
            f"{fraud_result['confidence']:.1%}",
            fraud_result['meta']['explain_time_ms'],
            len(fraud_result['explanation']['nodes']),
            len(fraud_result['explanation']['edges'])
        ]
    }
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    
    print("\n🔍 KEY INSIGHTS:")
    prob_diff = fraud_result['prediction_prob'] - legit_result['prediction_prob']
    print(f"   • Fraud probability difference: {prob_diff:.1%}")
    print(f"   • Model successfully distinguished between transaction types")
    print(f"   • Explanation graphs provide interpretable risk factors")
else:
    print("⚠️ Cannot perform comparison - missing prediction results")

## Service Metrics

In [ ]:
if service_available:
    print("📊 Fetching service performance metrics...\n")
    
    metrics = get_service_metrics()
    
    if metrics:
        print("⚡ SERVICE PERFORMANCE:")
        print(f"   Total Predictions: {metrics['total_predictions']}")
        print(f"   Average Response Time: {metrics['avg_response_time_ms']:.1f} ms")
        print(f"   Service Uptime: {metrics['uptime_seconds']:.1f} seconds")
        print(f"   Last Prediction: {metrics['last_prediction_time']}")
        
        if metrics['total_predictions'] > 0:
            print(f"\n📈 MODEL STATISTICS:")
            print(f"   Fraud Detection Rate: {metrics.get('fraud_detection_rate', 'N/A')}")
            print(f"   Average Confidence: {metrics.get('avg_confidence', 'N/A')}")
else:
    print("⚠️ Service not available - cannot fetch metrics")

## Custom Transaction Testing

In [ ]:
# Interactive testing - modify this cell to test your own transactions
custom_transaction = {
    "user_id": "user_test_555",
    "merchant_id": "merchant_electronics_888",
    "device_id": "device_mobile_def", 
    "ip_address": "10.0.0.1",
    "timestamp": datetime.now().isoformat(),
    "amount": 299.99,
    "currency": "USD",
    "location": "San Francisco, CA",
    "context": {
        "merchant_category": "electronics",
        "is_weekend": False
    }
}

if service_available:
    print("🧪 Testing custom transaction...\n")
    
    custom_result = predict_fraud(custom_transaction)
    
    if custom_result:
        print(f"📊 CUSTOM TRANSACTION RESULTS:")
        print(f"   Amount: ${custom_transaction['amount']}")
        print(f"   Category: {custom_transaction['context']['merchant_category']}")
        print(f"   Prediction: {custom_result['predicted_label'].upper()}")
        print(f"   Fraud Probability: {custom_result['prediction_prob']:.1%}")
        print(f"   Confidence: {custom_result['confidence']:.1%}")
        
        if custom_result['prediction_prob'] > 0.5:
            print("\n⚠️  HIGH RISK TRANSACTION DETECTED")
        else:
            print("\n✅ LOW RISK TRANSACTION")
else:
    print("⚠️ Service not available - skipping custom test")

## Summary and Next Steps

In [ ]:
print("🎯 DEMO SUMMARY")
print("═" * 50)
print()
print("✅ Successfully demonstrated:")
print("   • Real-time fraud prediction API")
print("   • Explainable AI with subgraph analysis")
print("   • Interactive graph visualization")
print("   • Service health monitoring")
print("   • Performance metrics tracking")
print()
print("🚀 Next steps for production deployment:")
print("   • Add authentication and rate limiting")
print("   • Implement model versioning and A/B testing")
print("   • Set up monitoring and alerting")
print("   • Scale with containerization (Docker)")
print("   • Add real-time streaming capabilities")
print()
print("📚 Additional resources:")
print("   • Web interface: http://localhost:8000/static/index.html")
print("   • API documentation: http://localhost:8000/docs")
print("   • Health endpoint: http://localhost:8000/health")
print("   • Metrics endpoint: http://localhost:8000/metrics")